In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession

In [0]:
#configs = {
#  "fs.azure.account.auth.type": "CustomAccessToken",
#  "fs.azure.account.custom.token.provider.class": spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")
#}

# Optionally, you can add <directory-name> to the source URI of your mount point.
#dbutils.fs.mount(
#  source = "abfss://bronze-parquet-files@weiridatalake.dfs.core.windows.net/",
#  mount_point = "/mnt/bronze",
#  extra_configs = configs)


In [0]:
dbutils.fs.ls("/mnt/bronze")

[FileInfo(path='dbfs:/mnt/bronze/customers.parquet', name='customers.parquet', size=14877, modificationTime=1723753599000),
 FileInfo(path='dbfs:/mnt/bronze/employees.parquet', name='employees.parquet', size=2711, modificationTime=1723753600000),
 FileInfo(path='dbfs:/mnt/bronze/offices.parquet', name='offices.parquet', size=2362, modificationTime=1723753600000),
 FileInfo(path='dbfs:/mnt/bronze/orderdetails.parquet', name='orderdetails.parquet', size=41567, modificationTime=1723753599000),
 FileInfo(path='dbfs:/mnt/bronze/orders.parquet', name='orders.parquet', size=9837, modificationTime=1723753599000),
 FileInfo(path='dbfs:/mnt/bronze/payments.parquet', name='payments.parquet', size=8355, modificationTime=1723753599000),
 FileInfo(path='dbfs:/mnt/bronze/product_lines.parquet', name='product_lines.parquet', size=4023, modificationTime=1723753599000),
 FileInfo(path='dbfs:/mnt/bronze/products.parquet', name='products.parquet', size=17152, modificationTime=1723753600000)]

In [0]:
#TABELAS BRONZA PARA DELTA

#customers

parquet_customers_path = "dbfs:/mnt/bronze/customers/*.parquet"


@dlt.table(
comment= 'raw parquet from data lake',
    table_properties={"quality": "silver"})
def bronze_customers():

    return (spark.read.format("parquet").load(parquet_customers_path))

ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 642, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/dlt/helpers.py", line 31, in call
    res = self.func()
          ^^^^^^^^^^^
  File "/root/.ipykernel/1098/command-400138607482050-2283703241", line 13, in bronze_customers
    return (spark.read.format("parquet").load(parquet_customers_path))
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/pyspark/instrumentation_utils.py", line 47, in wrapper
    res = func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/pyspark/sql/readwriter.py", line 312, in load
    return self._df(self._jreader.load(path))
       

py4j.Py4JException: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 642, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/dlt/helpers.py", line 31, in call
    res = self.func()
          ^^^^^^^^^^^
  File "/root/.ipykernel/1098/command-400138607482050-2283703241", line 13, in bronze_customers
    return (spark.read.format("parquet").load(parquet_customers_path))
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/pyspark/instrumentation_utils.py", line 47, in wrapper
    res = func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/pyspark/sql/readwriter.py", line 312, in load
    return self._df(self._jreader.load(path))
                    ^^^^^^

In [0]:
#orders

parquet_orders_path = "dbfs:/mnt/bronze/orders/*.parquet"

@dlt.table(
comment= 'raw parquet from data lake',
    table_properties={"quality": "silver"})
def bronze_orders():
    return (spark.read.format("parquet").load(parquet_orders_path))

ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 642, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/dlt/helpers.py", line 31, in call
    res = self.func()
          ^^^^^^^^^^^
  File "/root/.ipykernel/1098/command-400138607482051-4146287839", line 9, in bronze_orders
    return (spark.read.format("parquet").load(parquet_orders_path))
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/pyspark/instrumentation_utils.py", line 47, in wrapper
    res = func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/pyspark/sql/readwriter.py", line 312, in load
    return self._df(self._jreader.load(path))
                 

py4j.Py4JException: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 642, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/dlt/helpers.py", line 31, in call
    res = self.func()
          ^^^^^^^^^^^
  File "/root/.ipykernel/1098/command-400138607482051-4146287839", line 9, in bronze_orders
    return (spark.read.format("parquet").load(parquet_orders_path))
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/pyspark/instrumentation_utils.py", line 47, in wrapper
    res = func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/pyspark/sql/readwriter.py", line 312, in load
    return self._df(self._jreader.load(path))
                    ^^^^^^^^^^^^^^^^

In [0]:
#TRANSFORMAÇÕES COM VIEWS - CAMADA SILVER


@dlt.view(comment="sanitize data.")
def customers_enriched():
    return (
    dlt.read("bronze_customers")
    .select(
        col("customer_number").alias("user_id"),
        col("country").alias("country_name")
    )
)

ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 642, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/dlt/helpers.py", line 31, in call
    res = self.func()
          ^^^^^^^^^^^
  File "/root/.ipykernel/1098/command-400138607482053-833398461", line 7, in customers_enriched
    dlt.read("bronze_customers")
  File "/databricks/spark/python/dlt/api.py", line 605, in read
    pipeline.instance.get_scala_pipeline().read(name),
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1355, in __call__
    return_value = get_return_value(
                   ^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/pyspark

py4j.Py4JException: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 642, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/dlt/helpers.py", line 31, in call
    res = self.func()
          ^^^^^^^^^^^
  File "/root/.ipykernel/1098/command-400138607482053-833398461", line 7, in customers_enriched
    dlt.read("bronze_customers")
  File "/databricks/spark/python/dlt/api.py", line 605, in read
    pipeline.instance.get_scala_pipeline().read(name),
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1355, in __call__
    return_value = get_return_value(
                   ^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/pyspark/errors/exceptions/

In [0]:
@dlt.view(comment="sanitize data.")
def orders_enriched():
    return (
    dlt.read("bronze_orders")
    .select(
        col("order_number").alias("order_id"),
        col("customer_number").alias("order_user_id"),
        col("shipped_date"),
        col("order_date")
    )
)

ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 642, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/dlt/helpers.py", line 31, in call
    res = self.func()
          ^^^^^^^^^^^
  File "/root/.ipykernel/1098/command-400138607482054-2719045694", line 4, in orders_enriched
    dlt.read("bronze_orders")
  File "/databricks/spark/python/dlt/api.py", line 605, in read
    pipeline.instance.get_scala_pipeline().read(name),
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1355, in __call__
    return_value = get_return_value(
                   ^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/pyspark/erro

py4j.Py4JException: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 642, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/dlt/helpers.py", line 31, in call
    res = self.func()
          ^^^^^^^^^^^
  File "/root/.ipykernel/1098/command-400138607482054-2719045694", line 4, in orders_enriched
    dlt.read("bronze_orders")
  File "/databricks/spark/python/dlt/api.py", line 605, in read
    pipeline.instance.get_scala_pipeline().read(name),
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1355, in __call__
    return_value = get_return_value(
                   ^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/pyspark/errors/exceptions/captu

In [0]:
# CRIAÇÃO DAS TABELAS GOLDS

@dlt.table(
    comment="Country orders cleaned table",
    table_properties={"quality": "gold"}
)
def gold_country_orders():
    bronze_customers = dlt.read("customers_enriched")
    bronze_orders = dlt.read("orders_enriched")

    # Join datasets using an inner join
    join_country_orders = bronze_orders.join(
        bronze_customers,
        bronze_orders.order_user_id == bronze_customers.user_id,
        how='inner'
    )

    result_df = join_country_orders.select(
        "order_id",
        "order_user_id",
        "shipped_date",
        "order_date",
        "user_id",
        "country_name"
    )

    return final_query_df

ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 642, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/dlt/helpers.py", line 31, in call
    res = self.func()
          ^^^^^^^^^^^
  File "/root/.ipykernel/1098/command-400138607482063-2888877845", line 8, in gold_country_orders
    bronze_customers = dlt.read("customers_enriched")
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/dlt/api.py", line 605, in read
    pipeline.instance.get_scala_pipeline().read(name),
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1355, in __call__
    return_value = get_return_value(


py4j.Py4JException: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 642, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/dlt/helpers.py", line 31, in call
    res = self.func()
          ^^^^^^^^^^^
  File "/root/.ipykernel/1098/command-400138607482063-2888877845", line 8, in gold_country_orders
    bronze_customers = dlt.read("customers_enriched")
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/dlt/api.py", line 605, in read
    pipeline.instance.get_scala_pipeline().read(name),
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1355, in __call__
    return_value = get_return_value(
                   